In [1]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer

In [2]:
qual_cols = { "age", "TSH", "T3", "TT4", "T4U", "FTI", "TBG" }
cat_cols = { "sex", "on_thyroxine", "on_antithyroid_meds", "sick", "pregnant", "thyroid_surgery", "I131_treatment", "lithium", "goitre", "tumor", "psych", "referral_source", "target" }

In [3]:
remove_qual_cols = { "TBG" }
remove_cat_cols = set()

In [4]:
k = 10
neighbours = 2
iters = 10000

In [5]:
seed = 42

In [6]:
random_state = np.random.RandomState(seed)
random_generator = np.random.default_rng(seed)

In [7]:
sel_qual_cols = list(qual_cols.difference(remove_qual_cols))
sel_cat_cols = list(cat_cols.difference(remove_cat_cols))
display(sel_qual_cols)
display(sel_cat_cols)

['FTI', 'TSH', 'T3', 'TT4', 'age', 'T4U']

['thyroid_surgery',
 'sex',
 'goitre',
 'psych',
 'sick',
 'tumor',
 'pregnant',
 'on_thyroxine',
 'on_antithyroid_meds',
 'I131_treatment',
 'target',
 'referral_source',
 'lithium']

In [8]:
df = pd.read_csv("./data/thyroidDF.csv")
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,NaN,f,NaN,f,NaN,f,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,128.0,f,NaN,f,NaN,f,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,...,64.0,t,0.83,t,77.0,f,NaN,SVI,-,870119022
9168,22,M,f,f,f,f,f,f,f,f,...,91.0,t,0.92,t,99.0,f,NaN,SVI,-,870119023
9169,69,M,f,f,f,f,f,f,f,f,...,113.0,t,1.27,t,89.0,f,NaN,SVI,I,870119025
9170,47,F,f,f,f,f,f,f,f,f,...,75.0,t,0.85,t,88.0,f,NaN,other,-,870119027


In [9]:
df.dropna()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
167,40,F,f,f,f,f,f,f,f,f,...,3.9,t,0.83,t,5.0,t,28.0,other,F,840827019
5256,35,F,f,f,f,f,f,t,f,f,...,73.0,t,1.16,t,63.0,t,37.0,other,-,851128040
6044,77,F,f,f,f,f,f,f,f,f,...,120.0,t,0.96,t,124.0,t,45.0,SVI,-,860305064
6045,73,M,f,f,f,f,f,f,f,f,...,89.0,t,0.74,t,119.0,t,24.0,SVI,-,860305065
6747,77,F,f,f,f,f,f,f,f,f,...,131.0,t,1.04,t,126.0,t,25.0,SVI,K,860702030
6773,74,F,f,f,f,f,f,f,f,f,...,116.0,t,0.81,t,143.0,t,22.0,SVI,-,860703046
6862,60,M,f,f,f,f,f,f,f,f,...,92.0,t,0.84,t,110.0,t,21.0,other,-,860710043
6863,66,F,f,f,f,f,f,f,f,f,...,138.0,t,0.80,t,173.0,t,15.0,SVI,-,860710044
6880,42,F,f,f,f,f,f,f,f,f,...,106.0,t,0.98,t,108.0,t,27.0,other,-,860711039
6934,29,F,f,f,f,f,f,f,f,f,...,122.0,t,1.14,t,107.0,t,36.0,SVI,-,860717007


In [10]:
df[sel_qual_cols]

,FTI,TSH,T3,TT4,age,T4U
0,NaN,0.3,NaN,NaN,29,NaN
1,NaN,1.6,1.9,128.0,29,NaN
2,NaN,NaN,NaN,NaN,41,NaN
3,NaN,NaN,NaN,NaN,36,NaN
4,NaN,NaN,NaN,NaN,32,NaN
...,...,...,...,...,...,...
9167,77.0,NaN,NaN,64.0,56,0.83
9168,99.0,NaN,NaN,91.0,22,0.92
9169,89.0,NaN,NaN,113.0,69,1.27
9170,88.0,NaN,NaN,75.0,47,0.85


In [11]:
df[sel_cat_cols]

,thyroid_surgery,sex,goitre,psych,sick,tumor,pregnant,on_thyroxine,on_antithyroid_meds,I131_treatment,target,referral_source,lithium
0,f,F,f,f,f,f,f,f,f,f,-,other,f
1,f,F,f,f,f,f,f,f,f,f,-,other,f
2,f,F,f,f,f,f,f,f,f,f,-,other,f
3,f,F,f,f,f,f,f,f,f,f,-,other,f
4,f,F,f,f,f,f,f,f,f,f,S,other,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,f,M,f,f,f,f,f,f,f,f,-,SVI,f
9168,f,M,f,f,f,f,f,f,f,f,-,SVI,f
9169,f,M,f,f,f,f,f,f,f,f,I,SVI,f
9170,f,F,f,f,f,f,f,f,f,f,-,other,f


In [12]:
qual_na_df = df[sel_qual_cols].dropna()
qual_na_df

,FTI,TSH,T3,TT4,age,T4U
19,85.0,1.50,2.4,90.0,36,1.06
21,96.0,1.20,2.3,104.0,40,1.08
22,105.0,5.90,2.1,88.0,40,0.84
23,95.0,0.05,2.4,107.0,77,1.13
27,106.0,0.05,2.1,93.0,51,0.87
...,...,...,...,...,...,...
9129,94.0,0.73,1.8,85.0,65,0.90
9130,113.0,4.10,2.1,135.0,65,1.19
9134,39.0,53.00,1.0,49.0,74,1.25
9137,81.0,2.30,1.3,59.0,42,0.73


In [13]:
from data_split import k_fold_split

In [14]:
complete_vals_df, remove_vals_df = k_fold_split(qual_na_df, k, random_generator)
missing_vals_idxs = list(remove_vals_df.index)

real_df = pd.concat([complete_vals_df, remove_vals_df])

In [15]:
knn_imputed_df_map = dict()

for col in sel_qual_cols:
    missing_vals_df = remove_vals_df.copy()
    missing_vals_df[col] = np.NaN

    curr_df = pd.concat([complete_vals_df, missing_vals_df])

    knn_imputer = KNNImputer(n_neighbors=neighbours)
    knn_imputed_mat = knn_imputer.fit_transform(curr_df)
    
    knn_imputed_df = pd.DataFrame(knn_imputed_mat, columns=curr_df.columns, index=curr_df.index)
    knn_imputed_df["{} (real)".format(col)] = real_df[col]
    knn_imputed_df["{} (imputed)".format(col)] = knn_imputed_df[col]
    knn_imputed_df.drop([col], axis=1, inplace=True)

    knn_imputed_df = knn_imputed_df.loc[missing_vals_idxs]

    knn_imputed_df_map[col] = knn_imputed_df

In [16]:
for col, knn_imputed_df in knn_imputed_df_map.items():
    display("{} with KNN".format(col))
    display(knn_imputed_df)

'FTI with KNN'

,TSH,T3,TT4,age,T4U,FTI (real),FTI (imputed)
92,0.20,0.4,98.0,88.0,0.73,134.0,95.5
312,145.00,0.2,16.0,65.0,0.94,17.0,9.0
1347,0.10,2.2,153.0,33.0,0.81,188.0,123.5
4525,0.58,1.9,140.0,85.0,0.96,145.0,146.5
4328,0.33,2.3,99.0,74.0,0.98,101.0,90.5
...,...,...,...,...,...,...,...
7528,2.30,2.5,93.0,55.0,1.09,85.0,101.5
2110,1.80,2.0,66.0,42.0,0.93,71.0,73.5
3210,1.80,1.9,102.0,61.0,1.06,97.0,118.5
4874,1.80,2.4,73.0,90.0,1.07,69.0,66.5


'TSH with KNN'

,FTI,T3,TT4,age,T4U,TSH (real),TSH (imputed)
92,134.0,0.4,98.0,88.0,0.73,0.20,1.3000
312,17.0,0.2,16.0,65.0,0.94,145.00,55.0000
1347,188.0,2.2,153.0,33.0,0.81,0.10,0.2000
4525,145.0,1.9,140.0,85.0,0.96,0.58,1.3750
4328,101.0,2.3,99.0,74.0,0.98,0.33,0.6750
...,...,...,...,...,...,...,...
7528,85.0,2.5,93.0,55.0,1.09,2.30,3.1000
2110,71.0,2.0,66.0,42.0,0.93,1.80,0.6525
3210,97.0,1.9,102.0,61.0,1.06,1.80,0.8000
4874,69.0,2.4,73.0,90.0,1.07,1.80,1.6250


'T3 with KNN'

...

,FTI,TSH,T3,age,T4U,TT4 (real),TT4 (imputed)
92,134.0,0.20,0.4,88.0,0.73,98.0,114.5
312,17.0,145.00,0.2,65.0,0.94,16.0,24.5
1347,188.0,0.10,2.2,33.0,0.81,153.0,162.5
4525,145.0,0.58,1.9,85.0,0.96,140.0,135.0
4328,101.0,0.33,2.3,74.0,0.98,99.0,120.5
...,...,...,...,...,...,...,...
7528,85.0,2.30,2.5,55.0,1.09,93.0,78.5
2110,71.0,1.80,2.0,42.0,0.93,66.0,83.0
3210,97.0,1.80,1.9,61.0,1.06,102.0,93.5
4874,69.0,1.80,2.4,90.0,1.07,73.0,70.5


'age with KNN'

,FTI,TSH,T3,TT4,T4U,age (real),age (imputed)
92,134.0,0.20,0.4,98.0,0.73,88.0,36.5
312,17.0,145.00,0.2,16.0,0.94,65.0,78.5
1347,188.0,0.10,2.2,153.0,0.81,33.0,35.0
4525,145.0,0.58,1.9,140.0,0.96,85.0,39.0
4328,101.0,0.33,2.3,99.0,0.98,74.0,58.5
...,...,...,...,...,...,...,...
7528,85.0,2.30,2.5,93.0,1.09,55.0,57.5
2110,71.0,1.80,2.0,66.0,0.93,42.0,63.5
3210,97.0,1.80,1.9,102.0,1.06,61.0,46.5
4874,69.0,1.80,2.4,73.0,1.07,90.0,66.5


'T4U with KNN'

,FTI,TSH,T3,TT4,age,T4U (real),T4U (imputed)
92,134.0,0.20,0.4,98.0,88.0,0.73,0.745
312,17.0,145.00,0.2,16.0,65.0,0.94,1.020
1347,188.0,0.10,2.2,153.0,33.0,0.81,0.830
4525,145.0,0.58,1.9,140.0,85.0,0.96,0.965
4328,101.0,0.33,2.3,99.0,74.0,0.98,0.995
...,...,...,...,...,...,...,...
7528,85.0,2.30,2.5,93.0,55.0,1.09,1.070
2110,71.0,1.80,2.0,66.0,42.0,0.93,0.940
3210,97.0,1.80,1.9,102.0,61.0,1.06,1.050
4874,69.0,1.80,2.4,73.0,90.0,1.07,1.060


In [17]:
mice_imputed_df_map = dict()

for col in sel_qual_cols:
    missing_vals_df = remove_vals_df.copy()
    missing_vals_df[col] = np.NaN

    curr_df = pd.concat([complete_vals_df, missing_vals_df])

    mice_imputer = IterativeImputer(random_state=random_state, max_iter=iters)
    mice_imputed_mat = mice_imputer.fit_transform(curr_df)
    
    mice_imputed_df = pd.DataFrame(mice_imputed_mat, columns=curr_df.columns, index=curr_df.index)
    mice_imputed_df["{} (real)".format(col)] = real_df[col]
    mice_imputed_df["{} (imputed)".format(col)] = mice_imputed_df[col]
    mice_imputed_df.drop([col], axis=1, inplace=True)

    mice_imputed_df = mice_imputed_df.loc[missing_vals_idxs]

    mice_imputed_df_map[col] = mice_imputed_df

In [18]:
for col, mice_imputed_df in mice_imputed_df_map.items():
    display("{} with MICE".format(col))
    display(mice_imputed_df)

'FTI with MICE'

,TSH,T3,TT4,age,T4U,FTI (real),FTI (imputed)
92,0.20,0.4,98.0,88.0,0.73,134.0,129.278039
312,145.00,0.2,16.0,65.0,0.94,17.0,27.243706
1347,0.10,2.2,153.0,33.0,0.81,188.0,182.081947
4525,0.58,1.9,140.0,85.0,0.96,145.0,148.326582
4328,0.33,2.3,99.0,74.0,0.98,101.0,103.087148
...,...,...,...,...,...,...,...
7528,2.30,2.5,93.0,55.0,1.09,85.0,83.286508
2110,1.80,2.0,66.0,42.0,0.93,71.0,73.732414
3210,1.80,1.9,102.0,61.0,1.06,97.0,95.169255
4874,1.80,2.4,73.0,90.0,1.07,69.0,64.257721


'TSH with MICE'

,FTI,T3,TT4,age,T4U,TSH (real),TSH (imputed)
92,134.0,0.4,98.0,88.0,0.73,0.20,3.908273
312,17.0,0.2,16.0,65.0,0.94,145.00,26.829287
1347,188.0,2.2,153.0,33.0,0.81,0.10,-7.857079
4525,145.0,1.9,140.0,85.0,0.96,0.58,-2.840155
4328,101.0,2.3,99.0,74.0,0.98,0.33,6.274027
...,...,...,...,...,...,...,...
7528,85.0,2.5,93.0,55.0,1.09,2.30,11.137764
2110,71.0,2.0,66.0,42.0,0.93,1.80,12.320276
3210,97.0,1.9,102.0,61.0,1.06,1.80,9.200990
4874,69.0,2.4,73.0,90.0,1.07,1.80,15.730249


'T3 with MICE'

...

,FTI,TSH,T3,age,T4U,TT4 (real),TT4 (imputed)
92,134.0,0.20,0.4,88.0,0.73,98.0,96.748641
312,17.0,145.00,0.2,65.0,0.94,16.0,10.010594
1347,188.0,0.10,2.2,33.0,0.81,153.0,149.597693
4525,145.0,0.58,1.9,85.0,0.96,140.0,131.768495
4328,101.0,0.33,2.3,74.0,0.98,99.0,100.408736
...,...,...,...,...,...,...,...
7528,85.0,2.30,2.5,55.0,1.09,93.0,99.745641
2110,71.0,1.80,2.0,42.0,0.93,66.0,71.255371
3210,97.0,1.80,1.9,61.0,1.06,102.0,105.074988
4874,69.0,1.80,2.4,90.0,1.07,73.0,85.151604


'age with MICE'

,FTI,TSH,T3,TT4,T4U,age (real),age (imputed)
92,134.0,0.20,0.4,98.0,0.73,88.0,77.816599
312,17.0,145.00,0.2,16.0,0.94,65.0,77.754223
1347,188.0,0.10,2.2,153.0,0.81,33.0,77.833392
4525,145.0,0.58,1.9,140.0,0.96,85.0,77.822787
4328,101.0,0.33,2.3,99.0,0.98,74.0,77.809542
...,...,...,...,...,...,...,...
7528,85.0,2.30,2.5,93.0,1.09,55.0,77.805126
2110,71.0,1.80,2.0,66.0,0.93,42.0,77.799706
3210,97.0,1.80,1.9,102.0,1.06,61.0,77.808649
4874,69.0,1.80,2.4,73.0,1.07,90.0,77.799922


'T4U with MICE'

,FTI,TSH,T3,TT4,age,T4U (real),T4U (imputed)
92,134.0,0.20,0.4,98.0,88.0,0.73,0.747064
312,17.0,145.00,0.2,16.0,65.0,0.94,1.002598
1347,188.0,0.10,2.2,153.0,33.0,0.81,0.845666
4525,145.0,0.58,1.9,140.0,85.0,0.96,0.993857
4328,101.0,0.33,2.3,99.0,74.0,0.98,0.988771
...,...,...,...,...,...,...,...
7528,85.0,2.30,2.5,93.0,55.0,1.09,1.046764
2110,71.0,1.80,2.0,66.0,42.0,0.93,0.939074
3210,97.0,1.80,1.9,102.0,61.0,1.06,1.020298
4874,69.0,1.80,2.4,73.0,90.0,1.07,1.005513


In [19]:
from proba_utils import cum_sum_intervals_from_weights, sample_cols, create_nan_vals

In [20]:
weight_map = {
    "age" : 1,
    "T3"  : 1,
    "T4U" : 1,
    "TSH" : 1,
    "TT4" : 1,
    "FTI" : 1
}

cum_sum_intervals, cum_sum_col_map = cum_sum_intervals_from_weights(weight_map)

display(cum_sum_intervals)
display(cum_sum_col_map)

[Interval(0, 0.16666666666666666, closed='left'),
 Interval(0.16666666666666666, 0.3333333333333333, closed='left'),
 Interval(0.3333333333333333, 0.5, closed='left'),
 Interval(0.5, 0.6666666666666666, closed='left'),
 Interval(0.6666666666666666, 0.8333333333333333, closed='left'),
 Interval(0.8333333333333333, 1, closed='both')]

{0: 'age', 1: 'T3', 2: 'T4U', 3: 'TSH', 4: 'TT4', 5: 'FTI'}

In [21]:
picked_cols = sample_cols(remove_vals_df.shape[0], cum_sum_intervals, cum_sum_col_map, random_generator)
missing_vals_df, missing_col_map = create_nan_vals(remove_vals_df, picked_cols)

missing_vals_idxs = list(missing_vals_df.index)

real_df = pd.concat([complete_vals_df, remove_vals_df])
curr_df = pd.concat([complete_vals_df, missing_vals_df])

In [22]:
knn_imputer = KNNImputer(n_neighbors=neighbours)
knn_imputed_mat = knn_imputer.fit_transform(curr_df)

knn_imputed_df = pd.DataFrame(knn_imputed_mat, columns=real_df.columns, index=real_df.index)
knn_imputed_df = knn_imputed_df.loc[missing_vals_idxs]

for col in remove_vals_df.columns:
    knn_imputed_df["{} (real)".format(col)] = remove_vals_df[col]
    knn_imputed_df["{} (imputed)".format(col)] = knn_imputed_df[col]
    knn_imputed_df.drop([col], axis=1, inplace=True)

knn_imputed_df["imputed"] = picked_cols

knn_imputed_df

,FTI (real),FTI (imputed),TSH (real),TSH (imputed),T3 (real),T3 (imputed),TT4 (real),TT4 (imputed),age (real),age (imputed),T4U (real),T4U (imputed),imputed
92,134.0,134.0,0.20,0.20,0.4,0.4,98.0,112.0,88.0,88.0,0.73,0.730,TT4
312,17.0,17.0,145.00,145.00,0.2,0.2,16.0,16.0,65.0,65.0,0.94,1.020,T4U
1347,188.0,188.0,0.10,0.10,2.2,2.2,153.0,153.0,33.0,49.0,0.81,0.810,age
4525,145.0,145.0,0.58,0.58,1.9,1.9,140.0,140.0,85.0,85.0,0.96,0.965,T4U
4328,101.0,90.5,0.33,0.33,2.3,2.3,99.0,99.0,74.0,74.0,0.98,0.980,FTI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7528,85.0,85.0,2.30,2.30,2.5,2.5,93.0,100.5,55.0,55.0,1.09,1.090,TT4
2110,71.0,95.5,1.80,1.80,2.0,2.0,66.0,66.0,42.0,42.0,0.93,0.930,FTI
3210,97.0,136.0,1.80,1.80,1.9,1.9,102.0,102.0,61.0,61.0,1.06,1.060,FTI
4874,69.0,69.0,1.80,1.80,2.4,2.4,73.0,111.0,90.0,90.0,1.07,1.070,TT4


In [23]:
mice_imputer = IterativeImputer(random_state=random_state, max_iter=iters)
mice_imputed_mat = mice_imputer.fit_transform(curr_df)

mice_imputed_df = pd.DataFrame(mice_imputed_mat, columns=real_df.columns, index=real_df.index)
mice_imputed_df = mice_imputed_df.loc[missing_vals_idxs]

for col in remove_vals_df.columns:
    mice_imputed_df["{} (real)".format(col)] = remove_vals_df[col]
    mice_imputed_df["{} (imputed)".format(col)] = mice_imputed_df[col]
    mice_imputed_df.drop([col], axis=1, inplace=True)

mice_imputed_df["imputed"] = picked_cols

mice_imputed_df

,FTI (real),FTI (imputed),TSH (real),TSH (imputed),T3 (real),T3 (imputed),TT4 (real),TT4 (imputed),age (real),age (imputed),T4U (real),T4U (imputed),imputed
92,134.0,134.000000,0.20,0.200000,0.4,0.4,98.0,96.374866,88.0,88.000000,0.73,0.730000,TT4
312,17.0,17.000000,145.00,145.000000,0.2,0.2,16.0,16.000000,65.0,65.000000,0.94,1.008931,T4U
1347,188.0,188.000000,0.10,0.100000,2.2,2.2,153.0,153.000000,33.0,75.856907,0.81,0.810000,age
4525,145.0,145.000000,0.58,0.580000,1.9,1.9,140.0,140.000000,85.0,85.000000,0.96,0.994333,T4U
4328,101.0,102.878925,0.33,0.330000,2.3,2.3,99.0,99.000000,74.0,74.000000,0.98,0.980000,FTI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7528,85.0,85.000000,2.30,2.300000,2.5,2.5,93.0,100.145852,55.0,55.000000,1.09,1.090000,TT4
2110,71.0,73.360176,1.80,1.800000,2.0,2.0,66.0,66.000000,42.0,42.000000,0.93,0.930000,FTI
3210,97.0,94.890911,1.80,1.800000,1.9,1.9,102.0,102.000000,61.0,61.000000,1.06,1.060000,FTI
4874,69.0,69.000000,1.80,1.800000,2.4,2.4,73.0,85.643590,90.0,90.000000,1.07,1.070000,TT4
